In [18]:
import pandas as pd
import numpy as np
import os
import scipy
from scipy.interpolate import griddata
import torch
import torch.nn as nn
from tqdm import tqdm
import torch.nn.functional as F
from torch.nn.functional import mse_loss
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt

In [19]:
def compute_distance(x, y, d, n):
    """Compute the minimum distance of (x, y) from the nearest circular post."""
    tilt = 0.4/n
    centers = [(0, 0), (0, 0.4), (0.4, tilt), (0.4, 0.4+tilt)]
    
    distances = torch.full_like(x, float("inf"))  # Initialize distances with large values

    for cx, cy in centers:
        r = d / 2
        distance_to_post = torch.sqrt((x - cx) ** 2 + (y - cy) ** 2) - r
        distances = torch.minimum(distances, distance_to_post)  # Take the minimum distance

    return torch.maximum(distances, torch.tensor(0.0))  # Set negative distances to zero

def smooth_distance_to_posts(x, y, d, n, alpha=128.0, beta=50.0):
    """
    Smooth, differentiable approximation to the clamped min distance to 4 circular posts.

    Parameters
    ----------
    x, y, d, n : [N,1] tensors (requires_grad OK)
        n is used to compute tilt = 0.4 / n.
    alpha : float
        Softmin sharpness. Larger -> closer to true min. Try 32–128.
    beta : float
        Softplus sharpness. Larger -> closer to ReLU. Try 10–50.

    Returns
    -------
    dist_mask : [N,1] tensor >= 0
        Smoothly zero inside posts; smoothly ramps outside.
    """
    tilt = 0.4 / n  # [N,1]

    # Centers, broadcast to [N,4]
    cx = torch.cat([
        torch.zeros_like(x),            # x=0  (post 1)
        torch.zeros_like(x),            # x=0  (post 2)
        torch.full_like(x, 0.4),        # x=0.4(post 3)
        torch.full_like(x, 0.4)         # x=0.4(post 4)
    ], dim=1)

    cy = torch.cat([
        torch.zeros_like(y),            # y=0
        torch.full_like(y, 0.4),        # y=0.4
        tilt,                           # y=tilt
        0.4 + tilt                      # y=0.4+tilt
    ], dim=1)

    r = (d / 2).expand_as(cx)           # [N,4]

    # Signed distance to each post (neg inside)
    dist_each = torch.sqrt((x - cx)**2 + (y - cy)**2 + 1e-12) - r  # [N,4]

    # Softmin across posts
    # scale inputs to control sharpness; detach to avoid overflow hazard
    dist_softmin = -torch.logsumexp(-alpha * dist_each, dim=1, keepdim=True) / alpha  # [N,1]

    # Smooth clamp >=0
    dist_mask = F.softplus(dist_softmin, beta=beta)  # ~ReLU(dist_softmin)

    return dist_mask


In [20]:
# Folder path containing CSV files
folder_path = "data/csv"

# Define value ranges
first_values = np.arange(0.3, 0.7, 0.05)  # 0.25 to 0.7 with step 0.05
# second_values = np.arange(3, 15, 1)  # 3 to 14
second_values = [10]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [21]:
# Iterate over all possible files
for first in first_values:
    for second in second_values:
        first_str = f"{first:.2f}".rstrip("0").rstrip(".")
        file_name = f"W_{first_str}_{second}_1.csv"
        file_path = os.path.join(folder_path, file_name)
        
        if os.path.exists(file_path):
            print(f"Processing {file_name}...")
            
            # Load CSV file
            df = pd.read_csv(file_path)
            
            # Extract input and output data
            input_data = df.iloc[:, [0, 1, 5, 7]].values  # (x, y, d, N) [0, 1, 5, 7]
            output_data = df.iloc[:, [2, 3, 4]].values  # (u, v, p)
            
            x = torch.tensor(input_data[:, 0], dtype=torch.float32, device=device).view(-1, 1)
            y = torch.tensor(input_data[:, 1], dtype=torch.float32, device=device).view(-1, 1)
            d = torch.tensor(input_data[:, 2], dtype=torch.float32, device=device).view(-1, 1)

            # n = input_tensor[:, 3]
            n = torch.full_like(x, 10)  # Assuming n is constant for this example
            
            # Compute distance
            distances_1 = compute_distance(x, y, d, n)
            distances_2 = smooth_distance_to_posts(x, y, d, n)

            x_np = x.cpu().numpy().ravel()
            y_np = y.cpu().numpy().ravel()
            d1_np = distances_1.cpu().numpy().ravel()
            d2_np = distances_2.cpu().numpy().ravel()

            # plot the distances
            vmax = max(d1_np.max(), d2_np.max())

            plt.figure(figsize=(10, 5))
            plt.subplot(1, 2, 1)
            plt.scatter(x_np, y_np, c=d1_np, cmap='viridis', s=1, vmin=0, vmax=vmax)
            plt.colorbar(label='Distance')
            plt.title('Distance to Posts (compute_distance)')
            plt.xlabel('x')
            plt.ylabel('y')
            plt.subplot(1, 2, 2)
            plt.scatter(x_np, y_np, c=d2_np, cmap='viridis', s=1, vmin=0, vmax=vmax)
            plt.colorbar(label='Distance')
            plt.title('Distance to Posts (smooth_distance_to_posts)')
            plt.xlabel('x')
            plt.ylabel('y')
            plt.tight_layout()
            plt.savefig(f"distances_{first_str}_{second}.png")
            plt.close()
            print(f"Saved distances plot for {file_name}")



Processing W_0.3_10_1.csv...
Saved distances plot for W_0.3_10_1.csv
Processing W_0.35_10_1.csv...
Saved distances plot for W_0.35_10_1.csv
Processing W_0.4_10_1.csv...
Saved distances plot for W_0.4_10_1.csv
Processing W_0.45_10_1.csv...
Saved distances plot for W_0.45_10_1.csv
Processing W_0.5_10_1.csv...
Saved distances plot for W_0.5_10_1.csv
Processing W_0.55_10_1.csv...
Saved distances plot for W_0.55_10_1.csv
Processing W_0.6_10_1.csv...
Saved distances plot for W_0.6_10_1.csv
Processing W_0.65_10_1.csv...
Saved distances plot for W_0.65_10_1.csv
